## BIOINFORMATICS THESIS: MULTIMODAL NEURAL NETWORK

# CELL LINE: A549

In [17]:
import pandas as pd 
import numpy as np
import os

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict, OrderedDict
import pickle

import sqlite3
from sqlalchemy import create_engine

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [18]:
# create a database to store optuna studies with sqlite backend
#engine = create_engine('sqlite:///BIOINF_optuna_tuning.db')

In [19]:
from BIOINF_tesi.data_pipe import CELL_LINES, TASKS

In [20]:
cell_line = CELL_LINES[0]
cell_line

'A549'

---

In [21]:
from BIOINF_tesi.data_pipe import Load_Create_Task
from BIOINF_tesi.data_pipe import Build_DataLoader_Pipeline

In [7]:
from BIOINF_tesi.models import FFNN, CNN, CNN_LSTM
from BIOINF_tesi.models.utils import fit, Param_Search, Kfold_CV

In [17]:
with open ('results_dict.pickle', 'rb') as fin:
    results_dict = pickle.load(fin)
    results_dict = defaultdict(lambda: defaultdict(dict), results_dict)

## 1) ACTIVE ENHANCERS vs INACTIVE ENHANCERS

In [22]:
task = TASKS[0]
task

'active_E_vs_inactive_E'

In [9]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


### 1. FFNN

In [19]:
model=FFNN

In [11]:
# IMBALANCED
type_augm_genfeatures = 'smote'

kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=False,
                type_augm_genfeatures=type_augm_genfeatures,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_{type_augm_genfeatures}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_{type_augm_genfeatures}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:06:56,144] Using an existing study with name 'A549_active_E_vs_inactive_E_FFNN_smote_1' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`->`study.StudySummary`, `structs.FrozenTrial`->`trial.FrozenTrial`, `structs.TrialState`->`trial.TrialState`, `structs.TrialPruned`->`exceptions.TrialPruned`.
  warnings.warn(_message, FutureWarning)


Study statistics: 
  Number of finished trials:  5
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.08143502984111718
  Params: 
    dropout_l0: 0.4
    lr: 0.027111894800821008
    n_layers: 1
    n_units_l0: 128
    optimizer: Adam
    weight_decay: 0.0006750514030212153

===============> MODEL TESTING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:06:56,678] Using an existing study with name 'A549_active_E_vs_inactive_E_FFNN_smote_1_2' instead of creating a new one.


AUPRC test score: 0.10078988584259727


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.057860915492957736
  Params: 
    dropout_l0: 0.0
    dropout_l1: 0.4
    dropout_l2: 0.4
    dropout_l3: 0.5
    lr: 0.0069134605688635576
    n_layers: 4
    n_units_l0: 64
    n_units_l1: 64
    n_units_l2: 32
    n_units_l3: 32
    optimizer: Adam
    weight_decay: 0.004263230767303585

===============> MODEL TESTING
AUPRC test score: 0.05554865938430984


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:06:57,269] Using an existing study with name 'A549_active_E_vs_inactive_E_FFNN_smote_1_2_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.13661488396684307
  Params: 
    dropout_l0: 0.0
    dropout_l1: 0.3
    dropout_l2: 0.4
    lr: 0.017134120009264942
    n_layers: 3
    n_units_l0: 256
    n_units_l1: 32
    n_units_l2: 64
    optimizer: RMSprop
    weight_decay: 0.004466602964923941

===============> MODEL TESTING
AUPRC test score: 0.05852035749751737



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.07162


In [12]:
results_dict[cell_line][task][f'{model.__name__}_{type_augm_genfeatures}'] = kf_CV.scores_dict

In [13]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

In [14]:
# IMBALANCED
type_augm_genfeatures = 'double'

kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=False,
                type_augm_genfeatures=type_augm_genfeatures,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_{type_augm_genfeatures}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_{type_augm_genfeatures}_TEST.pt')

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:06:58,433] Using an existing study with name 'A549_active_E_vs_inactive_E_FFNN_double_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.11359246252835949
  Params: 
    dropout_l0: 0.2
    dropout_l1: 0.2
    dropout_l2: 0.0
    lr: 0.024731376245428784
    n_layers: 3
    n_units_l0: 64
    n_units_l1: 128
    n_units_l2: 16
    optimizer: RMSprop
    weight_decay: 0.004078330488802357

===============> MODEL TESTING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:06:58,692] Using an existing study with name 'A549_active_E_vs_inactive_E_FFNN_double_1_2' instead of creating a new one.


AUPRC test score: 0.05687437934458789


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.05786374654964966
  Params: 
    dropout_l0: 0.4
    dropout_l1: 0.3
    dropout_l2: 0.0
    lr: 5.65581253133464e-05
    n_layers: 3
    n_units_l0: 32
    n_units_l1: 64
    n_units_l2: 64
    optimizer: Adam
    weight_decay: 0.004385557345504528

===============> MODEL TESTING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:06:58,879] Using an existing study with name 'A549_active_E_vs_inactive_E_FFNN_double_1_2_3' instead of creating a new one.


AUPRC test score: 0.11023519722453504


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.1269234089835595
  Params: 
    dropout_l0: 0.4
    lr: 0.0013624976247216787
    n_layers: 1
    n_units_l0: 32
    optimizer: Adam
    weight_decay: 0.0009621113605341923

===============> MODEL TESTING
AUPRC test score: 0.13371450487634975



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.10027


In [15]:
results_dict[cell_line][task][f'{model.__name__}_{type_augm_genfeatures}'] = kf_CV.scores_dict

In [16]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
### 2. CNN

In [26]:
model=CNN

In [11]:
# IMBALANCED
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-25 18:37:43,397] Using an existing study with name 'A549_active_E_vs_inactive_E_CNN_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/opt/conda/lib/python3.7/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`->`study.StudySummary`, `structs.FrozenTrial`->`trial.FrozenTrial`, `structs.TrialState`->`trial.TrialState`, `structs.TrialPruned`->`exceptions.TrialPruned`.
  warnings.warn(_message, FutureWarning)


Study statistics: 
  Number of finished trials:  4
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.06671030052771904
  Params: 
    dropout_l0: 0.3
    dropout_l1: 0
    kernel_size_l0: 5
    kernel_size_l1: 15
    lr: 0.002492549875641731
    n_layers: 2
    optimizer: Nadam
    out_channels_l0: 16
    out_channels_l1: 96
    weight_decay: 0.05400548483737392

===============> MODEL TESTING
AUPRC test score: 0.056770109235352535




/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-25 18:37:46,521] Using an existing study with name 'A549_active_E_vs_inactive_E_CNN_1_2' instead of creating a new one.


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.07353027134563184
  Params: 
    dropout_l0: 0
    dropout_l1: 0.4
    kernel_size_l0: 15
    kernel_size_l1: 5
    lr: 0.003187423399350215
    n_layers: 2
    optimizer: Nadam
    out_channels_l0: 64
    out_channels_l1: 64
    weight_decay: 0.010152542998255033

===============> MODEL TESTING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-25 18:37:47,152] Using an existing study with name 'A549_active_E_vs_inactive_E_CNN_1_2_3' instead of creating a new one.


AUPRC test score: 0.06519933360703083


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.07250059734214663
  Params: 
    dropout_l0: 0.2
    dropout_l1: 0.4
    dropout_l2: 0.4
    dropout_l3: 0.5
    kernel_size_l0: 5
    kernel_size_l1: 11
    kernel_size_l2: 15
    kernel_size_l3: 5
    lr: 0.0010533905245534373
    n_layers: 4
    optimizer: Nadam
    out_channels_l0: 16
    out_channels_l1: 64
    out_channels_l2: 96
    out_channels_l3: 512
    weight_decay: 0.07991688720405098

===============> MODEL TESTING
AUPRC test score: 0.05836395233366432



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.06011


In [12]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [13]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

## 2) ACTIVE PROMOTERS vs INACTIVE PROMOTERS

In [12]:
task = TASKS[1]
task

'active_P_vs_inactive_P'

In [18]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---

### 1. FFNN

In [32]:
model=FFNN

In [20]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:07:03,471] Using an existing study with name 'A549_active_P_vs_inactive_P_FFNN_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.24630239801613507
  Params: 
    dropout_l0: 0.0
    lr: 0.010822302474528958
    n_layers: 1
    n_units_l0: 256
    optimizer: RMSprop
    weight_decay: 0.0037445355021975568

===============> MODEL TESTING
AUPRC test score: 0.24068126121120859


>>> ITERATION N. 2


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:07:03,696] Using an existing study with name 'A549_active_P_vs_inactive_P_FFNN_1_2' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.2496577966428925
  Params: 
    dropout_l0: 0.2
    dropout_l1: 0.2
    lr: 0.0003444879502087814
    n_layers: 2
    n_units_l0: 128
    n_units_l1: 16
    optimizer: RMSprop
    weight_decay: 0.009321065770065128

===============> MODEL TESTING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:07:03,921] Using an existing study with name 'A549_active_P_vs_inactive_P_FFNN_1_2_3' instead of creating a new one.


AUPRC test score: 0.23681676820641315


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.10209689281117858
  Params: 
    dropout_l0: 0.0
    dropout_l1: 0.4
    dropout_l2: 0.4
    dropout_l3: 0.4
    lr: 0.0002866616626840838
    n_layers: 4
    n_units_l0: 64
    n_units_l1: 32
    n_units_l2: 64
    n_units_l3: 4
    optimizer: Adam
    weight_decay: 0.006557897177013657

===============> MODEL TESTING
AUPRC test score: 0.24753521221225544



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.24168


In [21]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [22]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
### 2. CNN

In [37]:
model=CNN

In [17]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-25 18:37:55,021] Using an existing study with name 'A549_active_P_vs_inactive_P_CNN_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.18273752343785848
  Params: 
    dropout_l0: 0.4
    dropout_l1: 0.4
    kernel_size_l0: 5
    kernel_size_l1: 11
    lr: 6.581272290135745e-05
    n_layers: 2
    optimizer: Adam
    out_channels_l0: 16
    out_channels_l1: 64
    weight_decay: 0.0006137804736567615

===============> MODEL TESTING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-25 18:37:55,363] Using an existing study with name 'A549_active_P_vs_inactive_P_CNN_1_2' instead of creating a new one.


AUPRC test score: 0.19503897469253328


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.18840931902263708
  Params: 
    dropout_l0: 0.2
    dropout_l1: 0.4
    kernel_size_l0: 5
    kernel_size_l1: 11
    lr: 1.7628967520980997e-05
    n_layers: 2
    optimizer: RMSprop
    out_channels_l0: 32
    out_channels_l1: 32
    weight_decay: 0.0006201936675350008

===============> MODEL TESTING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-25 18:37:55,811] Using an existing study with name 'A549_active_P_vs_inactive_P_CNN_1_2_3' instead of creating a new one.


AUPRC test score: 0.1907454524469494


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.18569023842685717
  Params: 
    dropout_l0: 0.2
    dropout_l1: 0
    kernel_size_l0: 11
    kernel_size_l1: 15
    lr: 0.0011673180152785819
    n_layers: 2
    optimizer: Adam
    out_channels_l0: 32
    out_channels_l1: 96
    weight_decay: 0.012785315175687432

===============> MODEL TESTING
AUPRC test score: 0.19543550008918104



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.19374


In [18]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [19]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

## 3) ACTIVE ENHANCERS vs ACTIVE PROMOTERS

In [41]:
task = TASKS[2]
task

'active_E_vs_active_P'

In [24]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### 1. FFNN

In [42]:
model=FFNN

In [26]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:07:08,391] Using an existing study with name 'A549_active_E_vs_active_P_FFNN_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.7166961657151243
  Params: 
    dropout_l0: 0.2
    dropout_l1: 0.3
    lr: 0.0008497074355355139
    n_layers: 2
    n_units_l0: 64
    n_units_l1: 32
    optimizer: RMSprop
    weight_decay: 0.00042750672867661294

===============> MODEL TESTING
AUPRC test score: 0.6895415141808625


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:07:08,561] Using an existing study with name 'A549_active_E_vs_active_P_FFNN_1_2' instead of creating a new one.
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:07:08,677] Using an existing study with name 'A549_active_E_vs_active_P_FFNN_1_2_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.6473271526808818
  Params: 
    dropout_l0: 0.2
    dropout_l1: 0.0
    dropout_l2: 0.4
    lr: 0.007660288906168663
    n_layers: 3
    n_units_l0: 128
    n_units_l1: 32
    n_units_l2: 4
    optimizer: Adam
    weight_decay: 0.0011567097743807203

===============> MODEL TESTING
AUPRC test score: 0.727450023846858


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.6407601539356916
  Params: 
    dropout_l0: 0.3
    dropout_l1: 0.0
    lr: 0.00015428408333952455
    n_layers: 2
    n_units_l0: 32
    n_units_l1: 32
    optimizer: Nadam
    weight_decay: 0.02186164909249316

===============> MODEL TESTING
AUPRC test score: 0.6277023073733831



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.68156


In [27]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [28]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
### 2. CNN

In [48]:
model=CNN

In [23]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-25 18:38:00,600] Using an existing study with name 'A549_active_E_vs_active_P_CNN_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5907730514883093
  Params: 
    dropout_l0: 0
    dropout_l1: 0
    kernel_size_l0: 15
    kernel_size_l1: 5
    lr: 0.0002075881270271377
    n_layers: 2
    optimizer: RMSprop
    out_channels_l0: 64
    out_channels_l1: 96
    weight_decay: 0.0022148936628602002

===============> MODEL TESTING
AUPRC test score: 0.6564181982824108




/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-25 18:38:01,476] Using an existing study with name 'A549_active_E_vs_active_P_CNN_1_2' instead of creating a new one.


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.6022678317974148
  Params: 
    dropout_l0: 0
    kernel_size_l0: 11
    lr: 0.0005568882336287384
    n_layers: 1
    optimizer: RMSprop
    out_channels_l0: 16
    weight_decay: 0.007168711522114368

===============> MODEL TESTING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-25 18:38:01,877] Using an existing study with name 'A549_active_E_vs_active_P_CNN_1_2_3' instead of creating a new one.


AUPRC test score: 0.6782589514404268


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.6304423926366542
  Params: 
    dropout_l0: 0
    dropout_l1: 0
    kernel_size_l0: 15
    kernel_size_l1: 11
    lr: 0.0009214251171952463
    n_layers: 2
    optimizer: RMSprop
    out_channels_l0: 64
    out_channels_l1: 96
    weight_decay: 0.00030578141855603757

===============> MODEL TESTING
AUPRC test score: 0.6385317881614837



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.65774


In [24]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [25]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

## 4) INACTIVE ENHANCERS vs INACTIVE PROMOTERS

In [53]:
task = TASKS[3]
task

'inactive_E_vs_inactive_P'

In [30]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### 1. FFNN

In [54]:
model=FFNN

In [32]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:07:13,948] Using an existing study with name 'A549_inactive_E_vs_inactive_P_FFNN_1' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5745674373714987
  Params: 
    dropout_l0: 0.2
    lr: 5.008317052796455e-05
    n_layers: 1
    n_units_l0: 128
    optimizer: Adam
    weight_decay: 0.00020326126474239946

===============> MODEL TESTING
AUPRC test score: 0.573199921595868


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:07:14,291] Using an existing study with name 'A549_inactive_E_vs_inactive_P_FFNN_1_2' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.4002612818136451
  Params: 
    dropout_l0: 0.0
    dropout_l1: 0.0
    lr: 0.05242370181027648
    n_layers: 2
    n_units_l0: 256
    n_units_l1: 64
    optimizer: Nadam
    weight_decay: 0.09411938478516767

===============> MODEL TESTING
AUPRC test score: 0.39936786105460803


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:07:14,736] Using an existing study with name 'A549_inactive_E_vs_inactive_P_FFNN_1_2_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.566402609611672
  Params: 
    dropout_l0: 0.2
    dropout_l1: 0.4
    lr: 1.104105076424251e-05
    n_layers: 2
    n_units_l0: 32
    n_units_l1: 16
    optimizer: RMSprop
    weight_decay: 0.0003154095452807073

===============> MODEL TESTING
AUPRC test score: 0.5672468849646535



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.51327


In [33]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [34]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
### 2. CNN

In [59]:
model=CNN

In [29]:
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-25 18:38:07,012] Using an existing study with name 'A549_inactive_E_vs_inactive_P_CNN_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5760663763670382
  Params: 
    dropout_l0: 0.4
    dropout_l1: 0.4
    kernel_size_l0: 5
    kernel_size_l1: 5
    lr: 0.00037378373926965676
    n_layers: 2
    optimizer: RMSprop
    out_channels_l0: 32
    out_channels_l1: 32
    weight_decay: 0.0019414154771545153

===============> MODEL TESTING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-25 18:38:07,625] Using an existing study with name 'A549_inactive_E_vs_inactive_P_CNN_1_2' instead of creating a new one.


AUPRC test score: 0.5864192948037895


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5639809297465003
  Params: 
    dropout_l0: 0
    dropout_l1: 0.4
    dropout_l2: 0
    kernel_size_l0: 11
    kernel_size_l1: 5
    kernel_size_l2: 5
    lr: 0.0008113426985655316
    n_layers: 3
    optimizer: Nadam
    out_channels_l0: 64
    out_channels_l1: 96
    out_channels_l2: 96
    weight_decay: 0.05459213463084904

===============> MODEL TESTING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-25 18:38:08,122] Using an existing study with name 'A549_inactive_E_vs_inactive_P_CNN_1_2_3' instead of creating a new one.


AUPRC test score: 0.5719893161752271


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5888828656642322
  Params: 
    dropout_l0: 0.2
    kernel_size_l0: 11
    lr: 3.0731683605684234e-05
    n_layers: 1
    optimizer: Adam
    out_channels_l0: 32
    weight_decay: 0.014016142066702372

===============> MODEL TESTING
AUPRC test score: 0.5942429027008511



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.58422


In [30]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [31]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

## 5) ACTIVE ENHANCERS + ACTIVE PROMOTERS vs INACTIVE REST

In [8]:
task = TASKS[4]
task

'active_EP_vs_inactive_rest'

In [36]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### 1. FFNN

In [9]:
model=FFNN

In [38]:
# IMBALANCED
type_augm_genfeatures = 'smote'

kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=False,
                type_augm_genfeatures=type_augm_genfeatures,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_{type_augm_genfeatures}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_{type_augm_genfeatures}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:07:19,645] Using an existing study with name 'A549_active_EP_vs_inactive_rest_FFNN_smote_1' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.21535957579289228
  Params: 
    dropout_l0: 0.4
    dropout_l1: 0.3
    dropout_l2: 0.0
    lr: 0.001651837888927492
    n_layers: 3
    n_units_l0: 64
    n_units_l1: 32
    n_units_l2: 32
    optimizer: Nadam
    weight_decay: 0.0015060098914122845

===============> MODEL TESTING
AUPRC test score: 0.21303423144813163


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:07:23,538] Using an existing study with name 'A549_active_EP_vs_inactive_rest_FFNN_smote_1_2' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.21718298679155643
  Params: 
    dropout_l0: 0.0
    lr: 0.0019073114038689288
    n_layers: 1
    n_units_l0: 128
    optimizer: Nadam
    weight_decay: 0.00016946403036089493

===============> MODEL TESTING
AUPRC test score: 0.20922790198786095


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:07:28,970] Using an existing study with name 'A549_active_EP_vs_inactive_rest_FFNN_smote_1_2_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.2198683219329508
  Params: 
    dropout_l0: 0.0
    dropout_l1: 0.3
    dropout_l2: 0.0
    dropout_l3: 0.4
    lr: 0.0019100681965982071
    n_layers: 4
    n_units_l0: 256
    n_units_l1: 128
    n_units_l2: 16
    n_units_l3: 16
    optimizer: RMSprop
    weight_decay: 0.0005802275484579843

===============> MODEL TESTING
AUPRC test score: 0.20377105632873682



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.20868


In [39]:
results_dict[cell_line][task][f'{model.__name__}_{type_augm_genfeatures}'] = kf_CV.scores_dict

In [40]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

In [41]:
# IMBALANCED
type_augm_genfeatures = 'double'

kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=False,
                type_augm_genfeatures=type_augm_genfeatures,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_{type_augm_genfeatures}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_{type_augm_genfeatures}_TEST')

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:07:32,057] Using an existing study with name 'A549_active_EP_vs_inactive_rest_FFNN_double_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.2018778874464189
  Params: 
    dropout_l0: 0.4
    dropout_l1: 0.0
    dropout_l2: 0.0
    lr: 3.781803941584326e-05
    n_layers: 3
    n_units_l0: 32
    n_units_l1: 64
    n_units_l2: 32
    optimizer: Adam
    weight_decay: 0.000333225560015162

===============> MODEL TESTING
AUPRC test score: 0.21236833529824256


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:07:32,350] Using an existing study with name 'A549_active_EP_vs_inactive_rest_FFNN_double_1_2' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.21006128981487612
  Params: 
    dropout_l0: 0.2
    dropout_l1: 0.4
    dropout_l2: 0.0
    lr: 4.769799031853187e-05
    n_layers: 3
    n_units_l0: 64
    n_units_l1: 128
    n_units_l2: 4
    optimizer: RMSprop
    weight_decay: 0.00014174160214194557

===============> MODEL TESTING
AUPRC test score: 0.20921430575346636


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-04 07:07:32,681] Using an existing study with name 'A549_active_EP_vs_inactive_rest_FFNN_double_1_2_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.20398065072386948
  Params: 
    dropout_l0: 0.0
    dropout_l1: 0.2
    dropout_l2: 0.4
    lr: 1.2005247227684579e-05
    n_layers: 3
    n_units_l0: 128
    n_units_l1: 32
    n_units_l2: 16
    optimizer: Nadam
    weight_decay: 0.0009092847518404929

===============> MODEL TESTING
AUPRC test score: 0.20387677608140506



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.20849


In [42]:
results_dict[cell_line][task][f'{model.__name__}_{type_augm_genfeatures}'] = kf_CV.scores_dict

In [43]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
### 2. CNN

In [68]:
model=CNN

In [38]:
# IMBALANCED
kf_CV = Kfold_CV()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                sequence=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-25 18:38:25,279] Using an existing study with name 'A549_active_EP_vs_inactive_rest_CNN_1' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.14372508208904314
  Params: 
    dropout_l0: 0.2
    dropout_l1: 0.5
    dropout_l2: 0.5
    kernel_size_l0: 5
    kernel_size_l1: 15
    kernel_size_l2: 15
    lr: 0.00011225256307637438
    n_layers: 3
    optimizer: RMSprop
    out_channels_l0: 32
    out_channels_l1: 64
    out_channels_l2: 64
    weight_decay: 0.022471260724093313

===============> MODEL TESTING
AUPRC test score: 0.15808317888679663


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-25 18:38:25,991] Using an existing study with name 'A549_active_EP_vs_inactive_rest_CNN_1_2' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.15414174923759058
  Params: 
    dropout_l0: 0.4
    dropout_l1: 0.5
    dropout_l2: 0.5
    dropout_l3: 0.4
    kernel_size_l0: 11
    kernel_size_l1: 15
    kernel_size_l2: 15
    kernel_size_l3: 15
    lr: 0.0011837942258440628
    n_layers: 4
    optimizer: Adam
    out_channels_l0: 32
    out_channels_l1: 96
    out_channels_l2: 64
    out_channels_l3: 128
    weight_decay: 0.0007216288878670173

===============> MODEL TESTING
AUPRC test score: 0.15534093185276715


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models.py:362: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-25 18:38:26,540] Using an existing study with name 'A549_active_EP_vs_inactive_rest_CNN_1_2_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  6
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.16006682488349847
  Params: 
    dropout_l0: 0.2
    dropout_l1: 0.5
    dropout_l2: 0.5
    dropout_l3: 0.5
    kernel_size_l0: 15
    kernel_size_l1: 15
    kernel_size_l2: 15
    kernel_size_l3: 5
    lr: 0.00034665706509859866
    n_layers: 4
    optimizer: Adam
    out_channels_l0: 16
    out_channels_l1: 32
    out_channels_l2: 256
    out_channels_l3: 512
    weight_decay: 0.006522553452181734

===============> MODEL TESTING
AUPRC test score: 0.12690914560097605



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.14678


In [39]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [40]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

In [29]:
for model in ['FFNN','CNN','EmbraceNetMultimodal', 'ConcatNetMultimodal', 'EmbraceNetMultimodal_augmentation'] :
    print(f'\n{model}')
    for task in TASKS:
        print(f'\n{task}')
        for cell_line in CELL_LINES:
            print(f'\n{cell_line}')
            for n_iteration in range(3):
                n_iteration+=1
                l = torch.load(f'{cell_line}_{model}_{task}_{n_iteration}_test_.pt', map_location=device)
                
                for key in l['model_params'].keys():
                    if key.startswith(('n_units', 'n_layers', 'out_channels', 'kernel_size',
                                     'CNN_kernel_size', 'CNN_n_layers', 'CNN_out_channels',
                                     'FFNN_n_layers', 'FFNN_n_units', 'n_post_layers', 'EMBRACENET_embracement_size',
                                     'EMBRACENET_n_units', 'CONCATNET_n_post_layers', 'CONCATNET_n_units')):
                        l['model_params'][key]=int(l['model_params'][key])
                        
                torch.save(l, f'{cell_line}_{model}_{task}_{n_iteration}_test_.pt')


FFNN

active_E_vs_inactive_E

A549

GM12878

H1

HEK293

HEPG2

K562

MCF7

active_P_vs_inactive_P

A549

GM12878

H1

HEK293

HEPG2

K562

MCF7

active_E_vs_active_P

A549

GM12878

H1

HEK293

HEPG2

K562

MCF7

inactive_E_vs_inactive_P

A549

GM12878

H1

HEK293

HEPG2

K562

MCF7

active_EP_vs_inactive_rest

A549

GM12878

H1

HEK293

HEPG2

K562

MCF7

CNN

active_E_vs_inactive_E

A549

GM12878

H1

HEK293

HEPG2

K562

MCF7

active_P_vs_inactive_P

A549

GM12878

H1

HEK293

HEPG2

K562

MCF7

active_E_vs_active_P

A549

GM12878

H1

HEK293

HEPG2

K562

MCF7

inactive_E_vs_inactive_P

A549

GM12878

H1

HEK293

HEPG2

K562

MCF7

active_EP_vs_inactive_rest

A549

GM12878

H1

HEK293

HEPG2

K562

MCF7

EmbraceNetMultimodal

active_E_vs_inactive_E

A549

GM12878

H1

HEK293

HEPG2

K562

MCF7

active_P_vs_inactive_P

A549

GM12878

H1

HEK293

HEPG2

K562

MCF7

active_E_vs_active_P

A549

GM12878

H1

HEK293

HEPG2

K562

MCF7

inactive_E_vs_inactive_P

A549

GM12878

H1

HEK29

In [30]:
def get_difference_from_pval_dict(pval_dict):
    
    counter_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0))))
    
    for task in pval_dict.keys():
        for cell_line in pval_dict[task].keys():
            for fold in pval_dict[task][cell_line].keys():
                for b_model in pval_dict[task][cell_line][fold].keys():
                    for c_model in pval_dict[task][cell_line][fold][b_model].keys():

                        if pval_dict[task][cell_line][fold][b_model][c_model] >=0.05:
                            counter_dict[task][cell_line][b_model][c_model]+=0
                        else:
                            counter_dict[task][cell_line][b_model][c_model]+=1
                
             
    for task in counter_dict.keys():
        print(f'\n\n{task}')
        for cell_line in counter_dict[task].keys():
            print(f'\n\n{cell_line}')
            for b_model in counter_dict[task][cell_line].keys():
                print(f'\n\nBASE MODEL: {b_model}\n')
                for c_model in counter_dict[task][cell_line][b_model].keys():
                    
                    if counter_dict[task][cell_line][b_model][c_model] >=2:
                        print(f'{c_model} ===> different: True')
                    else:
                        print(f'{c_model} ===> different: False')

In [31]:
task=TASKS[4]

with open (f'pval_results_dict_{task}.pickle', 'rb') as fin:
    pval_dict = pickle.load(fin)
    pval_dict = defaultdict(lambda: defaultdict(dict), pval_dict)

In [32]:
pval_dict_2 = get_difference_from_pval_dict(pval_dict)



active_EP_vs_inactive_rest


A549


BASE MODEL: EmbraceNetMultimodal

FFNN ===> different: True
CNN ===> different: True
ConcatNetMultimodal ===> different: True
EmbraceNetMultimodal_augmentation ===> different: True


BASE MODEL: EmbraceNetMultimodal_augmentation

FFNN ===> different: True
CNN ===> different: True
ConcatNetMultimodal ===> different: True
EmbraceNetMultimodal ===> different: True


GM12878


BASE MODEL: EmbraceNetMultimodal

FFNN ===> different: True
CNN ===> different: True
ConcatNetMultimodal ===> different: True
EmbraceNetMultimodal_augmentation ===> different: True


BASE MODEL: EmbraceNetMultimodal_augmentation

FFNN ===> different: True
CNN ===> different: True
ConcatNetMultimodal ===> different: True
EmbraceNetMultimodal ===> different: True


H1


BASE MODEL: EmbraceNetMultimodal

FFNN ===> different: True
CNN ===> different: True
ConcatNetMultimodal ===> different: True
EmbraceNetMultimodal_augmentation ===> different: True


BASE MODEL: EmbraceNetMultimodal

In [9]:
with open ('results_dict.pickle', 'rb') as fin:
    results_dict = pickle.load(fin)
    results_dict = defaultdict(lambda: defaultdict(dict), results_dict)

In [28]:
results_dict['A549']['active_E_vs_inactive_E']['CNN']['iteration_n_1']['AUPRC_test'][-1]

0.056770109235352535

In [13]:
for cell in CELL_LINES:
    print(cell)
    for task in TASKS:
        print(task)
        for model in ['FFNN','CNN','ConcatNetMultimodal', 'EmbraceNetMultimodal', 'EmbraceNetMultimodal_augm']:
            print(model)
            print(results_dict[cell][task][model]['average_CV_AUPRC'])

A549
active_E_vs_inactive_E
FFNN
0.10027
CNN
0.06011
ConcatNetMultimodal
0.0996
EmbraceNetMultimodal
0.10933
EmbraceNetMultimodal_augm
0.09038
active_P_vs_inactive_P
FFNN
0.24168
CNN
0.19374
ConcatNetMultimodal
0.16468
EmbraceNetMultimodal
0.24523
EmbraceNetMultimodal_augm
0.20377
active_E_vs_active_P
FFNN
0.68156
CNN
0.65774
ConcatNetMultimodal
0.57216
EmbraceNetMultimodal
0.67179
EmbraceNetMultimodal_augm
0.67142
inactive_E_vs_inactive_P
FFNN
0.51327
CNN
0.58422
ConcatNetMultimodal
0.59152
EmbraceNetMultimodal
0.57709
EmbraceNetMultimodal_augm
0.58066
active_EP_vs_inactive_rest
FFNN
0.20868
CNN
0.14678
ConcatNetMultimodal
0.17485
EmbraceNetMultimodal
0.16442
EmbraceNetMultimodal_augm
0.20766
GM12878
active_E_vs_inactive_E
FFNN
0.42034
CNN
0.20147
ConcatNetMultimodal
0.37574
EmbraceNetMultimodal
0.3434
EmbraceNetMultimodal_augm
0.26883
active_P_vs_inactive_P
FFNN
0.34351
CNN
0.23723
ConcatNetMultimodal
0.29635
EmbraceNetMultimodal
0.34666
EmbraceNetMultimodal_augm
0.27028
active_E_vs_